## **Funciones para los Endpoints de API**

    1. Crearemos las funciones a partir de los datasets personalizados que creamos en el archivo 'Queries'

    2. Estas funciones seran utilizadas por nuestra API para dar respuesta a queries de nuestros endpoints y modelos de machine learning

    3. Los datasets que creamos para estas funcionesestan en el archivo 'Queries' con el objetivo de optimizar los tiempos de ejecucion y la utilizacion de recursos computacionales. 


### **Importamos las librerias** 

In [60]:
import pandas as pd
import os
import numpy as np

## **Endpoint 1**

#### **def PlayTimeGenre( genero : str ):** Debe devolver año con mas horas jugadas para dicho género.

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

Input: **PlayTimeGenre.csv**

In [21]:
def PlayTimeGenre(genero: str):
    result_df = pd.read_parquet('../Dataframes/Api_files/PlayTimeGenre.parquet')

    # Filtrar el DataFrame para el género específico
    filtered_df = result_df[result_df['genres'] == genero]
    
    # Agrupar por año de lanzamiento y sumar las horas jugadas
    grouped_df = filtered_df.groupby('year')['hours_game'].sum()
    
    # Encontrar el año con más horas jugadas
    max_hours_year = grouped_df.idxmax()

    # Construye el response_data
    response_data = {"Año de lanzamiento con más horas jugadas para {}: {}".format(genero, max_hours_year)}

    # Muestra el resultado
    return response_data

    

In [25]:
PlayTimeGenre('Action')

{'Año de lanzamiento con más horas jugadas para Action: 2012'}

## **Endpoint 2**

#### **def UserForGenre( genero : str ):** Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

Input: **UserForGenre.csv**

In [19]:
def UserForGenre(genero:str):
    consulta2 = pd.read_parquet('../Dataframes/Api_files/UserForGenre.parquet')
    
    # Filtrar el DataFrame por el género dado
    genre_data = consulta2[consulta2['genres'] == genero]

    # Encontrar al usuario con más horas jugadas para ese género
    top_user = genre_data.loc[genre_data['hours_game'].idxmax()]['user_id']

    # Crear una lista de acumulación de horas jugadas por año
    hours_by_year = genre_data.groupby('year')['hours_game'].sum().reset_index()
  
    hours_by_year = hours_by_year.rename(columns={'year': 'Año', 'hours_game': 'Horas'})
    
    hours_list = hours_by_year.to_dict(orient='records')

    # Crear el diccionario de retorno
    result = {
        "Usuario con más horas jugadas para Género {}".format(genero): top_user,
        "Horas jugadas": hours_list
    }

    return result

In [20]:
UserForGenre('Action')

{'Usuario con más horas jugadas para Género Action': 'Evilutional',
 'Horas jugadas': [{'Año': 1983, 'Horas': 5},
  {'Año': 1984, 'Horas': 1},
  {'Año': 1988, 'Horas': 11},
  {'Año': 1989, 'Horas': 8},
  {'Año': 1990, 'Horas': 23},
  {'Año': 1991, 'Horas': 12},
  {'Año': 1992, 'Horas': 10},
  {'Año': 1993, 'Horas': 30},
  {'Año': 1994, 'Horas': 92},
  {'Año': 1995, 'Horas': 169},
  {'Año': 1996, 'Horas': 124},
  {'Año': 1997, 'Horas': 111},
  {'Año': 1998, 'Horas': 1281},
  {'Año': 1999, 'Horas': 2357},
  {'Año': 2000, 'Horas': 5936},
  {'Año': 2001, 'Horas': 1232},
  {'Año': 2002, 'Horas': 211},
  {'Año': 2003, 'Horas': 8848},
  {'Año': 2004, 'Horas': 8229},
  {'Año': 2005, 'Horas': 1346},
  {'Año': 2006, 'Horas': 3908},
  {'Año': 2007, 'Horas': 1818},
  {'Año': 2008, 'Horas': 3163},
  {'Año': 2009, 'Horas': 6114},
  {'Año': 2010, 'Horas': 5209},
  {'Año': 2011, 'Horas': 6808},
  {'Año': 2012, 'Horas': 10588},
  {'Año': 2013, 'Horas': 6778},
  {'Año': 2014, 'Horas': 4275},
  {'Año': 2

## **Endpoint 3**

#### **def UsersRecommend( año : int )**: Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Input: **UsersRecommend.csv**

In [6]:
def UsersRecommend(year: int):
    df = pd.read_parquet('../Dataframes/Api_files/UsersRecommend.parquet')
    
    # Filtrar el DataFrame por el año especificado
    result_df = df[df['year'] == year]

    response_data = [{"Puesto 1 recomendados": result_df.iloc[0]['name']},
                     {"Puesto 2 recomendados": result_df.iloc[1]['name']},
                     {"Puesto 3 recomendados": result_df.iloc[2]['name']}]

    return response_data

In [7]:
UsersRecommend(2015)

[{'Puesto 1 recomendados': 'Counter-Strike: Global Offensive'},
 {'Puesto 2 recomendados': 'Team Fortress 2'},
 {'Puesto 3 recomendados': "Garry's Mod"}]

## **Endpoint 4**

#### def **UsersNotRecommend( año : int )**: Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Input: **UsersWorstDeveloper.csv**

In [8]:
def UsersNotRecommend(year: int):
    df = pd.read_parquet('../Dataframes/Api_files/UsersNotRecommend.parquet')

    # Filtrar el DataFrame por el año especificado
    result_df = df[df['year'] == year]

    response_data = [{"Puesto 1 Menos recomendado": result_df.iloc[0]['name']},
                     {"Puesto 2 Menos recomendado": result_df.iloc[1]['name']},
                     {"Puesto 3 Menos recomendado": result_df.iloc[2]['name']}]

    return response_data



In [9]:
UsersNotRecommend(2014)

[{'Puesto 1 Menos recomendado': 'Counter-Strike: Global Offensive'},
 {'Puesto 2 Menos recomendado': 'Team Fortress 2'},
 {'Puesto 3 Menos recomendado': 'DayZ'}]

## **Endpoint 5**

#### **def sentiment_analysis( año : int ):**:  Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

Input: **sentiment_analysis.csv**

In [50]:

def sentiment_analysis(year_released: int):
    df = pd.read_parquet('../Dataframes/Api_files/sentiment_analysis.parquet')

    # Filtrar por el año de release
    result_df = df[df['year_released'] == year_released]

    # Convertir las columnas que contienen numpy.int64 a tipos de Python nativos
    result_df['year_released'] = result_df['year_released'].astype(int)
    result_df['Negative'] = result_df['Negative'].astype(int)
    result_df['Neutral'] = result_df['Neutral'].astype(int)
    result_df['Positive'] = result_df['Positive'].astype(int)
    
    # Convertir a formato de diccionario
    response_data = result_df.set_index('year_released').to_dict(orient='index')
    
    return response_data


In [51]:
sentiment_analysis(2015)

/var/folders/jx/mgrpvh4n2f3b1l736sh02rpw0000gn/T/ipykernel_21430/1794004650.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['year_released'] = result_df['year_released'].astype(int)
/var/folders/jx/mgrpvh4n2f3b1l736sh02rpw0000gn/T/ipykernel_21430/1794004650.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Negative'] = result_df['Negative'].astype(int)
/var/folders/jx/mgrpvh4n2f3b1l736sh02rpw0000gn/T/ipykernel_21430/1794004650.py:10: SettingWithCopyWarning: 
A value is trying to b

{2015: {'Negative': 1100, 'Neutral': 572, 'Positive': 3549}}

## **ML model - sistema de recomendación item-item**

#### **def recomendacion_juego( id de producto ):** Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.


In [55]:
def recomendacion_usuario(item_id):
    df = pd.read_parquet('../Dataframes/ML_files/ItemItem_recomenda.parquet')
    
    # Filtrar el DataFrame por el año especificado
    result_df = df[df['item_id'] == item_id]
    
    response_data = result_df['Recomendaciones'].tolist()  # Convertir la serie a una lista
    
    return response_data

In [56]:
recomendacion_usuario(761140)

[array(['弹炸人2222', 'Uncanny Islands', 'Beach Rules',
        'Planetarium 2 - Zen Odyssey', 'The Warrior Of Treasures'],
       dtype=object)]

2a opcion

In [61]:
def recomendacion_usuario(item_id):
    df = pd.read_parquet('../Dataframes/ML_files/ItemItem_recomenda.parquet')
    
    # Filtrar el DataFrame por el item_id especificado
    result_df = df[df['item_id'] == item_id]
    
    # Convertir cualquier ndarray en la lista
    result_df['Recomendaciones'] = result_df['Recomendaciones'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
    
    response_data = result_df['Recomendaciones'].tolist()  # Convertir la serie a una lista
    
    return response_data


In [62]:
recomendacion_usuario(761140)

/var/folders/jx/mgrpvh4n2f3b1l736sh02rpw0000gn/T/ipykernel_21430/1402957408.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Recomendaciones'] = result_df['Recomendaciones'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)


[['弹炸人2222',
  'Uncanny Islands',
  'Beach Rules',
  'Planetarium 2 - Zen Odyssey',
  'The Warrior Of Treasures']]